In [15]:
import requests
import pandas as pd
import re
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [16]:
URL = "https://zarina.ru"
email = ''
password = ''

options = webdriver.ChromeOptions()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options = options)
driver.maximize_window()
driver.get(URL + '/catalog/clothes/')

authorization_btn = driver.find_element(By.XPATH, "//div[@class='header__icons-item_link']")
driver.execute_script("arguments[0].click();", authorization_btn)

driver.find_element(By.XPATH, "//input[@type='email']").send_keys(email)
driver.find_element(By.XPATH, "//input[@type='password']").send_keys(password)

enter_btn = driver.find_element(By.XPATH, "//button[@type='submit']")
driver.execute_script("arguments[0].click();", enter_btn)

time.sleep(5)
pagination_button = driver.find_element(By.XPATH, "//button[@class='pagination__more-btn']")

pages_cnt = 10
for _ in range(pages_cnt):
    try:
        driver.execute_script("arguments[0].click();", pagination_button)
    except Exception:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser') 
driver.close()

In [17]:
df = pd.DataFrame()

names = soup.find_all('div', class_ = 'catalog__product-title')
names = [name.a.text for name in names]
df['Name'] = names

prices = soup.find_all('div', class_ = 'catalog__product-price_current')
prices = [int(re.findall(r'\d+', price.text)[0]) for price in prices]
df['Price'] = prices


urls = soup.find_all('div', class_ = 'catalog__product-content')
pages = [requests.get(URL + url.a['href']) for url in urls]


descriptions = []
for page in pages:

    soup = BeautifulSoup(page.text, 'html.parser')
    full_description = ''
    
    color = soup.find('div', class_ = 'product__colors-title')
    full_description += 'Цвет: ' + color.text.strip() + '\n'

    description = soup.find('div', class_ = 'product__desc')

    for child in description.children:
        full_description += child.text.strip() + '\n'
    
    descriptions.append(full_description)
    
df['Description'] = descriptions

df.to_csv(r'.\zarina.csv', index = 0)
